# Uso da biblioteca SpaCy
### https://spacy.io/
### pip install -U spacy

In [64]:
import pandas as pd

## 1. Importando dados da preparação

In [65]:
PARQUET_MPO_PREPARADOS = '../../Data/Processed/mpo_preparados.parquet'

df_mpo = pd.read_parquet(PARQUET_MPO_PREPARADOS)

![SpaCy Pipeline](../../Docs/spacypipeline.png)

## Iniciando: Tokenização, índices e atributos léxicos

In [66]:
# Importe a classe do idioma portugues
from spacy.lang.pt import Portuguese

# Crie um objeto nlp
nlp = Portuguese()

# Processe o texto
doc = nlp("Olá meu nome é Marcelo e tenho 46 anos.")

# Imprima o texto do documento
print(doc.text)

# Imprima alguns atributos léxicos
for token in doc:
    print(f"id: {token.i}\t", f"token: {doc[token.i]}\t", f"pos_i: {token.idx}\t", f"String? {token.is_alpha}\t", f"Número? {token.is_digit}\t", f"Pontuação? {token.is_punct}")

Olá meu nome é Marcelo e tenho 46 anos.
id: 0	 token: Olá	 pos_i: 0	 String? True	 Número? False	 Pontuação? False
id: 1	 token: meu	 pos_i: 4	 String? True	 Número? False	 Pontuação? False
id: 2	 token: nome	 pos_i: 8	 String? True	 Número? False	 Pontuação? False
id: 3	 token: é	 pos_i: 13	 String? True	 Número? False	 Pontuação? False
id: 4	 token: Marcelo	 pos_i: 15	 String? True	 Número? False	 Pontuação? False
id: 5	 token: e	 pos_i: 23	 String? True	 Número? False	 Pontuação? False
id: 6	 token: tenho	 pos_i: 25	 String? True	 Número? False	 Pontuação? False
id: 7	 token: 46	 pos_i: 31	 String? False	 Número? True	 Pontuação? False
id: 8	 token: anos	 pos_i: 34	 String? True	 Número? False	 Pontuação? False
id: 9	 token: .	 pos_i: 38	 String? False	 Número? False	 Pontuação? True


## Importando pequeno modelo treinado em pt
## python -m spacy download pt_core_news_sm

In [87]:
import spacy

nlp = spacy.load("pt_core_news_sm")

In [68]:
# Processe o texto
doc = nlp("Olá meu nome é Marcelo e tenho 46 anos.")

# Imprima o texto do documento
print(doc.text)

Olá meu nome é Marcelo e tenho 46 anos.


## Observando previsão de classe gramatical

In [69]:
for token in doc:
    # Imprimir o texto e a classe gramatical prevista (pos_), marcador de dependência (dep_), índice da palavra principal (head) 
    print(token.text, token.pos_, token.dep_, token.head.text)

Olá VERB nsubj Marcelo
meu DET det nome
nome NOUN obj Olá
é AUX cop Marcelo
Marcelo PROPN ROOT Marcelo
e CCONJ cc tenho
tenho VERB conj Marcelo
46 NUM nummod anos
anos NOUN obj tenho
. PUNCT punct Marcelo


In [70]:
import spacy

spacy.explain("DET")


'determiner'

In [71]:
spacy.explain("CCONJ")

'coordinating conjunction'

In [72]:
spacy.explain("nummod")

'numeric modifier'

In [73]:
from spacy import displacy

In [75]:
displacy.render(doc, style="dep", jupyter = True)

## Observando as entidades nomeadas identificadas

In [76]:
# Iterar nas entidades previstas
for ent in doc.ents:
    # Imprimir o texto da entidade e seu marcador
    print(ent.text, ent.label_)

Olá MISC
Marcelo MISC


In [77]:
spacy.explain("MISC")

'Miscellaneous entities, e.g. events, nationalities, products or works of art'

In [79]:
doc = nlp("Olá meu nome é Marcelo, moro no Rio de Janeiro e trabalho na Microsoft.")

In [80]:
# Iterar nas entidades previstas
for ent in doc.ents:ORG
    # Imprimir o texto da entidade e seu marcador
    print(ent.text, ent.label_)

Olá MISC
Marcelo MISC
Rio de Janeiro LOC
Microsoft ORG


In [ ]:
spacy.explain("LOC")

In [ ]:
spacy.explain("ORG")

In [81]:
displacy.render(doc, style="ent")

## Usando Comparadores. Além de textos podem comparar atributos léxicos, lematização e outros

In [82]:
# Processe o texto
doc = nlp("Olá meu nome é Marcelo, moro no Rio de Janeiro e trabalho na Apple com iPhone X. Já trabalhei com iPhone 6 e nunca irei trabalhar com o iPhone 13.")

# Imprima o texto do documento
print(doc.text)

Olá meu nome é Marcelo, moro no Rio de Janeiro e trabalho na Apple com iPhone X. Já trabalhei com iPhone 6 e nunca irei trabalhar com o iPhone 13.


In [83]:
# Iterar nas entidades previstas
for ent in doc.ents:
    # Imprimir o texto da entidade e seu marcador
    print(ent.text, ent.label_)

Olá MISC
Marcelo MISC
Rio de Janeiro LOC
Apple ORG
iPhone X. MISC
iPhone MISC
iPhone MISC


In [84]:
displacy.render(doc, style="ent")

In [85]:
# Importar o Comparador (Matcher)
from spacy.matcher import Matcher

# Inicializar o comparador com o vocabulário. Obrigatório
matcher = Matcher(nlp.vocab)

# Adicionar a expressão ao comparador
pattern = [[{"TEXT": "iPhone"},{"IS_DIGIT":True}]
          ,[{"LOWER": "iphone"},{"LOWER":"x."}]
          ,[{"LEMMA": "trabalhar"}]]
ORG

matcher.add("IPHONE_PATTERN", pattern)

# Chamar o matcher no doc
matches = matcher(doc)

# Iterar nas correspondências
for match_id, start, end in matches:
    # Selecionar a partição que houve correspondência
    matched_span = doc[start:end]
    print(matched_span.text)

trabalho
iPhone X.
trabalhei
iPhone 6
trabalhar
iPhone 13


## Comparando textos por similaridade. Uso de word2Vec

### Somentes modelos _md ou _lg tem word to vector (python -m spacy download pt_core_news_lg)
### método default: similaridade de cossenos, mas pode ser ajustado
### Frases curtas são melhores que grandes com muitas palaras sem importância


In [102]:
nlp = spacy.load("pt_core_news_lg")

doc1 = nlp("Eu gosto de caminhar no parque")

doc2 = nlp("Eu gosto de praticar atividades ao ar livre")

print(doc1.similarity(doc2))

0.633895312399778


In [103]:
doc3 = nlp("Amarelo é mais bonito que Azul")

doc4 = nlp("Nas cores do arco íris o Roxo é o mais bonito")

print(doc1.similarity(doc2))

0.633895312399778


In [104]:
doc4 = nlp("Amarelo é mais bonito que Azul")

token1 = doc4[0]
token2 = doc4[5]

print(token1.similarity(token2))

0.66269594


### W2V em português parece não ser tão bom assim.
### Acessando o vetor para uma palavra qualquer

In [105]:
doc4[0].vector

array([-2.2635e-01,  1.4836e+00,  2.1009e+00, -3.2984e+00,  7.9271e-01,
       -1.3651e+00,  7.1088e-01, -1.5358e-01,  3.2697e+00,  2.4465e+00,
       -3.0082e-01,  5.0253e-02,  1.0965e+00,  5.8674e-01,  1.0070e+00,
       -2.4560e-01,  2.7139e+00, -1.5109e+00,  2.3853e+00,  1.2757e+00,
        1.9025e+00,  1.2140e-01, -1.0203e+00, -2.3683e+00,  1.5887e+00,
        8.4399e-01, -1.6584e+00, -7.2702e-01,  1.4246e+00,  1.8543e+00,
        1.1990e+00,  8.0773e-01,  1.1072e+00, -7.5184e-02,  3.0759e+00,
        2.2237e-01, -1.8473e+00,  7.1692e-01, -3.3460e-03,  5.7602e-01,
       -2.4090e+00, -8.6423e-01, -8.1773e-02, -3.6546e-02,  2.2307e+00,
       -1.2621e+00,  5.3564e-01, -1.4126e+00, -1.5432e+00, -1.2284e+00,
        3.3149e-01,  1.3002e+00,  4.2351e-01, -4.2618e-01, -7.4095e-01,
        1.0854e+00,  1.0451e+00,  2.4983e+00, -1.8599e+00,  3.7690e-02,
       -1.0240e+00,  1.3962e+00,  7.5540e-01,  9.6488e-01, -6.0423e-01,
       -6.2135e-01,  3.5685e-01, -6.3712e-01,  4.0014e-01,  2.15

In [107]:
doc5 = nlp("Este é um bom restaurante. Logo depois vamos a um excelente bar")

span1 = doc5[3:5]
span2 = doc5[10:13]

print(span1.similarity(span2), span1, span2)

0.58121794 bom restaurante um excelente bar


## Investigando o pipeline

In [108]:
# Print the names of the pipeline components
print(nlp.pipe_names)

# Print the full pipeline of (name, component) tuples
print(nlp.pipeline)

['tok2vec', 'morphologizer', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fab79d040e0>), ('morphologizer', <spacy.pipeline.morphologizer.Morphologizer object at 0x7fab79cefef0>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7fac18695220>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7fab87b9cd60>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7fab93a1b280>), ('lemmatizer', <spacy.pipeline.lemmatizer.Lemmatizer object at 0x7fab92229300>)]


### Componentes customizados podem ser incluídos no pipeline em qualquer posição. São chamados quando é instanciado o NLP

### É uma simples função que recebe o doc, faz algo com ele e o retorna. É importante sempre retornar o doc para que os outros pipelines possam utilizá-lo

<code>
import spacy

#### Define the custom component
def length_component(doc):
    # Get the doc's length
    doc_length = len(doc)
    print(f"This document is {doc_length} tokens long.")
    # Return the doc
    return doc


#### Load the small English model
nlp = spacy.load("en_core_web_sm")

#### Add the component first in the pipeline and print the pipe names
nlp.add_pipe(length_component, first=True)
print(nlp.pipe_names)

#### Process a text
doc = nlp("This is a sentence.")
<code>


## Extensão de atributos customizados

In [ ]:
from spacy.tokens import Span

nlp = spacy.load("pt_core_news_lg")


In [111]:

def get_wikipedia_url(span):
    # Get a Wikipedia URL if the span has one of the labels
    if span.label_ in ("PERSON", "ORG", "GPE", "LOC"):
        entity_text = span.text.replace(" ", "_")
        return "https://en.wikipedia.org/w/index.php?search=" + entity_text


# Set the Span extension wikipedia_url using the getter get_wikipedia_url
Span.set_extension("wikipedia_url", getter=get_wikipedia_url, force = True)

doc = nlp("Olá meu nome é Marcelo, moro no Rio de Janeiro e trabalho na Apple com iPhone X. Já trabalhei com iPhone 6 e nunca irei trabalhar com o iPhone 13.")

for ent in doc.ents:
    # Print the text and Wikipedia URL of the entity
    print(ent.text, ent._.wikipedia_url)

Marcelo None
Rio de Janeiro https://en.wikipedia.org/w/index.php?search=Rio_de_Janeiro
Apple https://en.wikipedia.org/w/index.php?search=Apple
iPhone None
iPhone None
iPhone None
